# INSTALL JULIA - COLAB

PEGUE O SCRIPT DE TEMPLATE [AQUI](https://colab.research.google.com/drive/1lLHnAUl1kqPB6Fc1tF9r19om24DbYYFh)!

In [ ]:
# %%shell
# set -e

# #---------------------------------------------------#
# JULIA_VERSION="1.10.4" # any version ≥ 0.7.0
# JULIA_PACKAGES="IJulia BenchmarkTools Plots"
# JULIA_NUM_THREADS=2
# #---------------------------------------------------#

# if [ -z `which julia` ]; then
# # Install Julia
# JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
# echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
# BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
# URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
# wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
# tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
# rm /tmp/julia.tar.gz

# # Install Packages
# for PKG in `echo $JULIA_PACKAGES`; do
#     echo "Installing Julia package $PKG..."
#     julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
# done

# # Install kernel and rename it to "julia"
# echo "Installing IJulia kernel..."
# julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
#     "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
# KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
# KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
# mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

# echo ''
# echo "Successfully installed `julia -v`!"
# echo "Please reload this page (press Ctrl+R or F5) then"
# echo "jump to the 'Checking the Installation' section."
# fi

In [ ]:
versioninfo()

# PROJETO 1 - ANALISE DE ALGORITMO

## LIBS & CONST

In [ ]:
using Plots
using BenchmarkTools
using Statistics
using Random

In [ ]:
#CONSTs
benchmark_count = 1
img_output = "dummy"

In [ ]:
# Constants
n_values = [10^i for i in 4:7]  # Vectors Sizes
q_values = [10^i for i in 2:5]  # Amount of Keys

In [ ]:
# Initialize the list of vectors and keys
key_list = [collect(1:q) for q in q_values]  # K2[1-100], ..., K5[1-10^5]
vector_list = [rand(1:10^5, n) for n in n_values]

In [ ]:
# Initialize the list of timers
time_simple = zeros(length(n_values), length(q_values))
time_optimized = zeros(length(n_values), length(q_values))
time_binary = zeros(length(n_values), length(q_values))
time_insetion = zeros(length(n_values))
time_merge = zeros(length(n_values))
time_quick = zeros(length(n_values))

## SEARCH FUNCTIONS

In [ ]:
# Simple linear search function
function linear_simple_search(vector, key)
    for i in 1:length(vector)
        if key == vector[i]
            return i  # Returns the index where the key was found
        end
    end
    return -1  # Returns -1 if the key is not found
end

In [ ]:
# Optimized linear search function
function linear_optimized_search(vector, key)
    for i in 1:length(vector)
        if key == vector[i]
            return i  # Returns the index where the key was found
        elseif key < vector[i]
            return -1  # Returns -1 if the key is not found
        end
    end
    return -1  # Returns -1 if the key is not found
end

In [ ]:
# Binary search function
function binary_search(vector, key)
    low, high = 1, length(vector)
    while low <= high
        mid = div(low + high, 2)
        if vector[mid] == key
            return mid  # Returns the index where the key was found
        elseif vector[mid] < key
            low = mid + 1
        else
            high = mid - 1
        end
    end
    return -1  # Returns -1 if the key is not found
end

## BENCHMARKING

In [ ]:
# Dummy sort function
function dummy_sort(time_matrix, sort_function)
    for i in 1:length(vector_list)   # For each vector size
        start = time_ns()
        for c in 1:benchmark_count
            sort_function!(vector_list[i])
        end
        time_matrix[i] = (time_ns() - start) / 1e9
    end
end

In [ ]:
# Dummy search function
function dummy_search(search_func, vector, keys)
    [search_func(vector, k) for k in keys]
end

In [ ]:
# Benchmark macro
function my_benchmark(search_func, time_matrix)
    for i in 1:length(vector_list)   # For each vector size
        for j in 1:length(key_list)  # For each key numbers
            start = time_ns()
            for c in 1:benchmark_count
                dummy_search(search_func, [vector_list[i]], [key_list[j]])
            end
            time_matrix[i, j] = (time_ns() - start) / 1e9
        end
    end
end

In [ ]:
my_benchmark(simple_search, time_simple)

In [ ]:
# Sorting the vectors - Benchmarking
dummy_sort(time_insetion, InsertionSort)
dummy_sort(time_merge, MergeSort)
dummy_sort(time_quick, QuickSort)

vector_list

In [ ]:
my_benchmark(optimized_search, time_optimized)

In [ ]:
my_benchmark(binary_search, time_binary)

## PLOTS

In [ ]:
# Plotting the results
xtick_positions = [0, 25000, 50000, 75000, 10^5]
xtick_labels = ["0", "25.000", "50.000", "75.000", "100.000"]

for i in 1:length(n_values)
    p = plot(
        q_values,
        time_simple[i, :],
        label = "Simples",
        xlabel = "Número de Busca (q)",
        ylabel = "Tempo (s)",
        title = "Tamanho do Vetor (n) = $(n_values[i])",
        legend = :topleft,
        xticks = (xtick_positions, xtick_labels),
        yscale = :log10,  # log scale
        lw = 2
    )
    plot!(q_values, time_optimized[i, :], label = "Otimizada", lw = 2)
    plot!(q_values, time_binary[i, :], label = "Binária", lw = 2)
    # plot!(q_values, time_sort[i] .* ones(length(q_values)), label = "Ordenação", lw = 2)

    display(p)
    savefig(p, "../imgs/n_size/log/$(img_output)_10^$(i+3).jpg")
end

In [ ]:
xtick_positions = [0, 25000, 50000, 75000, 10^5]
xtick_labels = ["0", "25.000", "50.000", "75.000", "100.000"]

# Ploting graphics
for i in 1:4
    p = plot(
        q_values,
        time_simple [i, :],
        label = "Simples",
        xlabel = "Numero de Busca (q)",
        ylabel = "Tempo (ns)",
        title = "Tamanho do Vetor (n) = $(n_values[i])",
        legend = :topleft,
        xticks = (xtick_positions, xtick_labels),
        lw = 2
    )
    plot!(q_values, time_optimized[i, :], label = "Otmizada", lw = 2)
    plot!(q_values, time_binary[i, :], label = "Binária", lw = 2)
    # plot!(q_values, time_sort, label = "Ordenarção", lw = 2)

    display(p)
    savefig(p, "../imgs/n_size/linear/$(img_output)_10^$(i+3).png")
end

In [ ]:
xtick_positions = [1, 2500000, 5000000, 7500000, 10^7]
xtick_labels = ["1", "2.500.000", "5.000.000", "7.500.000", "10^7"]

# Ploting graphics
for i in 1:4
    p = plot(
        n_values,
        time_simple[:, i],
        label = "Simples",
        xlabel = "Tamanho do Vetor (n)",
        ylabel = "Tempo (s)",
        title = "Numero de Busca (q) = $(q_values[i])",
        legend = :topleft,
        yscale = :log10,
        xticks = (xtick_positions, xtick_labels),
        lw = 2
    )
    plot!(n_values, time_optimized[:, i], label = "Otmizada", lw = 2)
    plot!(n_values, time_binary[:, i], label = "Binária", lw = 2)
    # plot!(n_values, time_sort, label = "Ordenarção", lw = 2)

    display(p)
    savefig(p, "../imgs/q_size/log/$(img_output)Tempo_10^$(i+1).png")
end

In [ ]:
xtick_positions = [10^5, 2500000, 5000000, 7500000, 10^7]
xtick_labels = ["100.000", "2.500.000", "5.000.000", "7.500.000", "10.000.000"]

# Ploting graphics
for i in 1:4
    p = plot(
        n_values,
        time_simple[:, i],
        label = "Simples",
        xlabel = "Tamanho do Vetor (n)",
        ylabel = "Tempo (ns)",
        title = "Numero de Busca (q) = $(q_values[i])",
        legend = :topright,
        xlims = (10^5, 10^7),
        xticks = (xtick_positions, xtick_labels),
        lw = 2
    )
    
    plot!(n_values, time_optimized[:, i], label = "Otmizada", lw = 2)
    plot!(n_values, time_binary[:, i], label = "Binária", lw = 2)
    # plot!(n_values, time_sort, label = "Ordenarção", lw = 2)


    display(p)
    savefig(p, "../imgs/q_size/linear/$(img_output)_10^$(i+1).png")
end